In [1]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=55d8c69332c3ba2b2405b99728f50576bfa0ec1839b1e5427f7b4f2233d4eab9
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [2]:
def black_box_function(x, y):
  return (4 - 2.1*(x**2) + (x**4)/3)*(x**2) + (x*y) + (-4 + (4*(y**2)))*(y**2)

In [3]:
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'x': (-3, 3), 'y': (-2, 2)}

optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    random_state=1,
)

In [4]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)


|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  1        | -0.265    | -0.4979   |  0.8813   |
|  2        |  110.1    | -2.999    | -0.7907   |
|  3        |  71.79    | -2.842    | -0.6775   |
|  4        |  105.6    | -2.973    | -1.082    |
|  5        |  3.61     | -2.104    |  0.5152   |


In [5]:
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

Iteration 0: 
	{'target': -0.2650082867644827, 'params': {'x': -0.4978679717845562, 'y': 0.8812979737686324}}
Iteration 1: 
	{'target': 110.13991869176739, 'params': {'x': -2.9993137510959307, 'y': -0.7906697094726409}}
Iteration 2: 
	{'target': 71.78927010059918, 'params': {'x': -2.8415417002388166, 'y': -0.6775081049627025}}
Iteration 3: 
	{'target': 105.61436929335453, 'params': {'x': -2.973470851997723, 'y': -1.082375009314677}}
Iteration 4: 
	{'target': 3.6103908488482563, 'params': {'x': -2.1041597209576226, 'y': 0.5152470560110505}}


In [6]:
optimizer.set_bounds(new_bounds={"x": (-2, 3)})

optimizer.maximize(
    init_points=0,
    n_iter=5,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  6        |  0.7717   | -1.57     |  1.035    |
|  7        |  44.62    | -1.989    | -1.895    |
|  8        |  150.9    |  3.0      | -2.0      |
|  9        |  113.2    |  3.0      | -1.43     |
|  10       |  57.11    |  2.342    | -2.0      |


In [7]:
optimizer.probe(
    params={"x": 0.5, "y": 0.7},
    lazy=True,
)

optimizer.probe(
    params=[-0.3, 0.1],
    lazy=True,
)

# Will probe only the two points specified above
optimizer.maximize(init_points=0, n_iter=0)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  11       |  0.2244   |  0.5      |  0.7      |
|  12       |  0.2736   | -0.3      |  0.1      |


In [8]:
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

In [9]:
logger = JSONLogger(path="./logs.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

# Results will be saved in ./logs.json
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  13       |  22.1     | -1.266    | -1.631    |
|  14       |  2.042    | -1.069    | -0.6178   |
|  15       |  107.6    |  3.0      | -0.3168   |
|  16       |  109.8    |  3.0      |  0.5887   |
|  17       |  8.357    |  2.203    |  0.2007   |


In [10]:
from bayes_opt.util import load_logs


new_optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds={"x": (-2, 2), "y": (-2, 2)},
    verbose=2,
    random_state=7,
)

# New optimizer is loaded with previously seen points
load_logs(new_optimizer, logs=["./logs.json"]);